In [ ]:
import numpy as np
from astropy.coordinates import EarthLocation, SkyCoord
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from matvis import conversions, simulate_vis
from pyuvsim.analyticbeam import AnalyticBeam
from astropy.time import Time
from bokeh.layouts import gridplot
import matplotlib.pyplot as plt
import astropy.units as u

output_notebook()

# Antenna positions for three antennas in a line from east to west (14 m baseline)
ants = {
    0: (0, 0, 0),
    1: (14, 0, 0),
    2: (28, 0, 0)
}
freqs = np.array([1e8, 2e8, 1e8])
# Define Gaussian beams with sigma 0.5
beams = [AnalyticBeam("gaussian", sigma=0.19, spectral_index=0, ref_freq=1e8)]
beam_idx = [0, 0, 0]  # All antennas are gaussian

# Frequency channels and LSTs
Nfreqs = 100
freqs = np.linspace(1e8, 2e8, Nfreqs)  # Frequencies in Hz

Ntimes = 24  # One observation per hour for 24 hours
lsts = np.linspace(0, 2 * np.pi, Ntimes)  # LSTs covering one full day

# Source coordinates and fluxes at the time of observation (at 100 MHz freq)
sources = [
    {'coords': SkyCoord(ra=0*u.deg, dec=-30.72152777777791*u.deg), 'flux': 2},
    {'coords': SkyCoord(ra=120*u.deg, dec=-30.72152777777791*u.deg), 'flux': 4},
    {'coords': SkyCoord(ra=240*u.deg, dec=-30.72152777777791*u.deg), 'flux': 6}
]

sp_index = 0  # Spectral index for all sources

# Flux of the sources at each frequency
fluxes = np.array([source['flux'] for source in sources])
flux_allfreq = ((freqs[:, np.newaxis] / freqs[0]) ** sp_index * fluxes).T

# Correct source locations for HERA
hera_lon = 21.428305555555557
hera_lat = -30.72152777777791
hera_height = 1073.0
location = EarthLocation.from_geodetic(lat=hera_lat, lon=hera_lon, height=hera_height)

obstime = Time("2024-08-05T00:00:00", format="isot", scale="utc")  # Starting time

ra = np.array([source['coords'].ra.rad for source in sources])
dec = np.array([source['coords'].dec.rad for source in sources])
ra_new, dec_new = conversions.equatorial_to_eci_coords(
    ra, dec, obstime, location, unit="rad", frame="icrs"
)

# Run the simulation
vis_vc = simulate_vis(
    ants=ants,
    fluxes=flux_allfreq,
    ra=ra_new,
    dec=dec_new,
    freqs=freqs,
    lsts=lsts,
    beams=beams,
    beam_idx=np.array(beam_idx),
    polarized=False,
    precision=2,
)

# Convert LST from radians to hours for plotting
lsts_hours = lsts * 12 / np.pi

colors = ['blue', 'green', 'red']
line_styles = ['solid', 'dashed', 'dotted']

# Plot auto correlation (amplitude and phase vs frequency) using Bokeh
auto_amp_freq_fig = figure(width=450, height=350, title="Auto-correlation Amplitude vs Frequency", x_axis_label='Frequency [MHz]', y_axis_label='Amplitude')
auto_phase_freq_fig = figure(width=450, height=350, title="Auto-correlation Phase vs Frequency", x_axis_label='Frequency [MHz]', y_axis_label='Phase [radians]')

for ant in range(3):
    auto_amp_freq_fig.line(freqs / 1.e6, np.abs(vis_vc[:, 0, ant, ant]), line_width=2, color=colors[ant], line_dash=line_styles[ant], legend_label=f'Auto {ant}')
    auto_phase_freq_fig.line(freqs / 1.e6, np.angle(vis_vc[:, 0, ant, ant]), line_width=2, color=colors[ant], line_dash=line_styles[ant], legend_label=f'Auto {ant}')
    auto_phase_freq_fig.y_range.start = -np.pi
    auto_phase_freq_fig.y_range.end = np.pi

auto_amp_freq_fig.legend.location = "top_right"
auto_phase_freq_fig.legend.location = "top_right"

cross_color_idx = 0
# Plot cross correlation (amplitude and phase vs frequency) using Bokeh
cross_amp_freq_fig = figure(width=450, height=350, title="Cross-correlation Amplitude vs Frequency", x_axis_label='Frequency [MHz]', y_axis_label='Amplitude')
cross_phase_freq_fig = figure(width=450, height=350, title="Cross-correlation Phase vs Frequency", x_axis_label='Frequency [MHz]', y_axis_label='Phase [radians]')

for ant1 in range(3):
    for ant2 in range(ant1 + 1, 3):
        cross_amp_freq_fig.line(freqs / 1.e6, np.abs(vis_vc[:, 0, ant1, ant2]), line_width=2, color=colors[cross_color_idx], line_dash=line_styles[cross_color_idx], legend_label=f'Cross ({ant1},{ant2})')
        cross_phase_freq_fig.line(freqs / 1.e6, np.angle(vis_vc[:, 0, ant1, ant2]), line_width=2, color=colors[cross_color_idx], line_dash=line_styles[cross_color_idx], legend_label=f'Cross ({ant1},{ant2})')
        cross_phase_freq_fig.y_range.start = -np.pi
        cross_phase_freq_fig.y_range.end = np.pi
        cross_color_idx += 1

cross_amp_freq_fig.legend.location = "top_right"
cross_phase_freq_fig.legend.location = "top_right"

# Plot auto correlation (amplitude and phase vs LST) using Bokeh
auto_amp_lsts_fig = figure(width=450, height=350, title="Auto-correlation Amplitude vs LST", x_axis_label='LST [hours]', y_axis_label='Amplitude')
auto_phase_lsts_fig = figure(width=450, height=350, title="Auto-correlation Phase vs LST", x_axis_label='LST [hours]', y_axis_label='Phase [radians]')

for ant in range(3):
    auto_amp_lsts_fig.line(lsts_hours, np.abs(vis_vc[0, :, ant, ant]), line_width=2, color=colors[ant], line_dash=line_styles[ant], legend_label=f'Auto {ant}')
    auto_phase_lsts_fig.line(lsts_hours, np.angle(vis_vc[0, :, ant, ant]), line_width=2, color=colors[ant], line_dash=line_styles[ant], legend_label=f'Auto {ant}')
    auto_phase_lsts_fig.y_range.start = -np.pi
    auto_phase_lsts_fig.y_range.end = np.pi

auto_amp_lsts_fig.legend.location = "top_right"
auto_phase_lsts_fig.legend.location = "top_right"

# Plot cross correlation (amplitude and phase vs LST) using Bokeh
cross_amp_lsts_fig = figure(width=450, height=350, title="Cross-correlation Amplitude vs LST", x_axis_label='LST [hours]', y_axis_label='Amplitude')
cross_phase_lsts_fig = figure(width=450, height=350, title="Cross-correlation Phase vs LST", x_axis_label='LST [hours]', y_axis_label='Phase [radians]')

cross_color_idx = 0
for ant1 in range(3):
    for ant2 in range(ant1 + 1, 3):
        cross_amp_lsts_fig.line(lsts_hours, np.abs(vis_vc[0, :, ant1, ant2]), line_width=2, color=colors[cross_color_idx], line_dash=line_styles[cross_color_idx], legend_label=f'Cross ({ant1},{ant2})')
        cross_phase_lsts_fig.line(lsts_hours, np.unwrap(np.angle(vis_vc[0, :, ant1, ant2])), line_width=2, color=colors[cross_color_idx], line_dash=line_styles[cross_color_idx], legend_label=f'Cross ({ant1},{ant2})')
        cross_color_idx += 1

cross_amp_lsts_fig.legend.location = "top_right"
cross_phase_lsts_fig.legend.location = "top_right"

# Create a gridplot for the auto-correlation and cross-correlation figures
grid = gridplot([
    [auto_amp_freq_fig, auto_phase_freq_fig],
    [cross_amp_freq_fig, cross_phase_freq_fig],
    [auto_amp_lsts_fig, auto_phase_lsts_fig],
    [cross_amp_lsts_fig, cross_phase_lsts_fig]
])

# Display the plot inline in the notebook
show(grid)

# Matplotlib plotting

# Create a meshgrid for frequency and LST
FREQ_grid, LST_grid = np.meshgrid(freqs / 1.e6, lsts_hours)

# Plotting auto-correlations for all antennas
for ant in range(3):
    # Extract the amplitude and phase data for auto-correlation
    amplitude_auto = np.abs(vis_vc[:, :, ant, ant])
    phase_auto = np.unwrap(np.angle(vis_vc[:, :, ant, ant]), axis=1)

    # Plot the amplitude heatmap for auto-correlation
    fig, ax = plt.subplots(1, 1, figsize=(16, 8))
    c = ax.pcolormesh(FREQ_grid, LST_grid, amplitude_auto.T, shading='auto')
    ax.set_xlabel('Frequency [MHz]', fontsize=15)
    ax.set_ylabel('LST [hours]', fontsize=15)
    ax.set_title(f'Amplitude of Auto-Correlation (Antenna {ant})', fontsize=18)
    fig.colorbar(c, ax=ax, label='Amplitude')
    plt.show()

    # Plot the phase heatmap for auto-correlation
    fig, ax = plt.subplots(1, 1, figsize=(16, 8))
    c = ax.pcolormesh(FREQ_grid, LST_grid, phase_auto.T, shading='auto', cmap='twilight')
    ax.set_xlabel('Frequency [MHz]', fontsize=15)
    ax.set_ylabel('LST [hours]', fontsize=15)
    ax.set_title(f'Phase of Auto-Correlation (Antenna {ant})', fontsize=18)
    fig.colorbar(c, ax=ax, label='Phase [radians]')
    plt.show()

# Plotting cross-correlations for all pairs of antennas
for ant1 in range(3):
    for ant2 in range(ant1 + 1, 3):
        # Extract the amplitude and phase data for cross-correlation
        amplitude_cross = np.abs(vis_vc[:, :, ant1, ant2])
        phase_cross = np.unwrap(np.angle(vis_vc[:, :, ant1, ant2]), axis=1)

        # Plot the amplitude heatmap for cross-correlation
        fig, ax = plt.subplots(1, 1, figsize=(16, 8))
        c = ax.pcolormesh(FREQ_grid, LST_grid, amplitude_cross.T, shading='auto')
        ax.set_xlabel('Frequency [MHz]', fontsize=15)
        ax.set_ylabel('LST [hours]', fontsize=15)
        ax.set_title(f'Amplitude of Cross-Correlation (Antenna {ant1}, {ant2})', fontsize=18)
        fig.colorbar(c, ax=ax, label='Amplitude')
        plt.show()

        # Plot the phase heatmap for cross-correlation
        fig, ax = plt.subplots(1, 1, figsize=(16, 8))
        c = ax.pcolormesh(FREQ_grid, LST_grid, phase_cross.T, shading='auto', cmap='twilight')
        ax.set_xlabel('Frequency [MHz]', fontsize=15)
        ax.set_ylabel('LST [hours]', fontsize=15)
        ax.set_title(f'Phase of Cross-Correlation (Antenna {ant1}, {ant2})', fontsize=18)
        fig.colorbar(c, ax=ax, label='Phase [radians]')
        plt.show()
